In [ ]:
import warnings
warnings.filterwarnings('ignore')

### [ TEST ] LLAMA Model 

In [4]:
from llama3 import LlamaModel

In [5]:
model = LlamaModel()
model.load_model()

Connecting to Ollama server
✅ Model llama3.1:latest loaded sucessfully via Ollama


In [6]:
model.generate_text(prompt= 'hom nay troi mua')

'Tình hình thời tiết hôm nay không có thông tin chính xác về vị trí của bạn. Tuy nhiên, tôi có thể cho bạn biết một số cách để tra cứu tình hình thời tiết tại nơi bạn đang ở.\n\n1. **Tra cứu trên Internet:** Bạn có thể tìm kiếm trên các trang web như weather.com, accuweather.com hoặc meteoblue.com để xem tình hình thời tiết hôm nay và dự báo trong ngày.\n\n2. **Tải ứng dụng thời tiết:** Có rất nhiều ứng dụng thời tiết miễn phí và trả phí cho điện thoại di động mà bạn có thể tải về và sử dụng. Một số ví dụ bao gồm Weather Underground, Dark Sky (sau khi được mua lại đã kết thúc hỗ trợ), The Weather Channel, hay AccuWeather.\n\n3. **Kênh thông tin thời tiết:** Các kênh truyền hình địa phương hoặc quốc gia cũng thường xuyên cung cấp thông tin thời tiết cho khu vực của bạn.\n\n4. **Thông báo từ chính quyền địa phương:** Trong trường hợp có bão, lũ lụt hoặc các tình huống khẩn cấp khác, chính quyền địa phương có thể gửi thông báo qua điện thoại di động, email hoặc các kênh truyền thông xã hộ

### [TEST] RAG Sytem 

In [2]:
from rag_system import RAGSystem

In [3]:
# Init the rag system 
menu_file_path = '../data/menu.json'

try: 
    rag = RAGSystem(menu_file_path)
    print(f'System status: {rag.get_stats()}')
except Exception as e: 
    print(f'Initialize failed: {e}')



Loaded 9 menu items
Loading embedding model all-MiniLM-L6-v2...
Building search index for menu items


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]

✅ Index built with 9 items
✅ RAG system initialized successfully.
System status: {'total_items': 9, 'model_name': 'all-MiniLM-L6-v2', 'embedding_dimension': 384, 'index_size': 9}


### [TEST] AI Waiter

In [2]:
from ai_chatbot import AIWaiter

/home/lequocthinh/miniconda3/envs/deeplearning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Test the complete AI Waiter system
def test_ai_waiter_complete():
    """Test the complete AI Waiter with RAG + Llama integration"""
    try:
        # Initialize AI Waiter
        print("🚀 Starting complete AI Waiter test...")
        waiter = AIWaiter(
            menu_file_path="../data/menu.json",
            model_name="llama3.1:latest"
        )
        
        # Show stats
        stats = waiter.get_stats()
        print(f"📊 System Stats: {stats}")
        
        print("\n🤖 Xin chào! Tôi là Linh, tôi sẽ giúp bạn chọn món hôm nay! 😊")
        
        # Test various queries
        test_queries = [
            "Xin chào, menu hôm nay có gì?",
            "Tôi muốn tìm món cay",
            "Có món chay nào không?", 
            "Gợi ý món ngon cho 2 người",
            "Món nào có giá khoảng 100k?",
            "Tôi thích hải sản, có món gì phù hợp?"
        ]
        
        for i, query in enumerate(test_queries, 1):
            print(f"\n{'='*60}")
            print(f"👤 Test {i}: {query}")
            print("="*60)
            
            response = waiter.chat(query)
            print(f"🤖 Linh: {response}")
            
            # Add small delay to see responses clearly
            import time
            time.sleep(1)
        
        # Save conversation
        waiter.save_conversation("ai_waiter_test_conversation.json")
        
        print("\n" + "="*60)
        print("✅ All tests completed successfully!")
        print("="*60)
        
    except Exception as e:
        print(f"❌ Test failed: {e}")
        import traceback
        traceback.print_exc()

# Run the test
test_ai_waiter_complete()

🚀 Starting complete AI Waiter test...
🤖 Initialize Chatbot...
📚 Loading RAG system...
Loaded 3 menu items
Loading embedding model all-MiniLM-L6-v2...
Building search index for menu items


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.66it/s]


✅ Index built with 3 items
✅ RAG system initialized successfully.
🦙 Loading Llama model...
Connecting to Ollama server
✅ Model llama3.1:latest loaded sucessfully via Ollama
✅ Chatbot initialization complete!
📊 System Stats: {'conversations': 0, 'model_name': 'llama3.1:latest', 'rag_stats': {'total_items': 3, 'model_name': 'all-MiniLM-L6-v2', 'embedding_dimension': 384, 'index_size': 3}}

🤖 Xin chào! Tôi là Linh, tôi sẽ giúp bạn chọn món hôm nay! 😊

👤 Test 1: Xin chào, menu hôm nay có gì?
🤖 Linh: Chào bạn! 😊 Chúc bạn có một bữa ăn ngon tại nhà hàng chúng tôi hôm nay!

Hôm nay, chúng tôi có nhiều món ăn hấp dẫn để lựa chọn.  Bạn thấy đấy, chúng ta có **Canh Nấm Hạt Sen (Chay)**, một món canh thanh đạm và bổ dưỡng với nấm đông cô, nấm bào ngư, hạt sen bùi bùi và cà rốt.  Giá của món này là 55.

👤 Test 2: Tôi muốn tìm món cay
🤖 Linh: Bạn đang tìm kiếm một món ăn có vị cay đấy! 🌶️

Ở nhà hàng chúng tôi có một số lựa chọn cho bạn. Món đầu tiên tôi muốn giới thiệu là **Đậu Hũ Sốt Nấm Tứ Xuyên